# Activation Checkpointing: A First Principles Deep DiveMemory is the bottleneck in deep learning. Not compute. Not data. Memory.This notebook tears apart activation checkpointing from first principles. You will understand exactly why activations dominate memory, how checkpointing trades compute for memory, and when this trade-off makes sense.---## The Core ProblemDuring training you do two things:1. **Forward pass**: compute outputs layer by layer2. **Backward pass**: compute gradients using the chain ruleHere's what kills you: backward needs intermediate values from forward. If forward does `y = sin(x)`, backward needs `x` to compute `dy/dx = cos(x)`. PyTorch keeps `x` alive until backward reaches that node. These are "saved tensors" in autograd-speak.So activation memory accumulates through forward and peaks at the start of backward. You've got this pile of "will need later" tensors sitting in VRAM, waiting.## What Checkpointing Actually DoesThink of training like hiking a trail:**Normal training**: You drop breadcrumbs at every step (store all activations). Easy to retrace (backprop), but you're carrying a giant breadcrumb bag (VRAM).**Checkpointing**: You only drop breadcrumbs at a few checkpoints. On the way back, when you need details between checkpoints, you re-walk that segment.That's literally it. Save fewer tensors in forward, recompute them on demand during backward. Memory goes down, compute goes up.## Concrete ExampleSuppose your forward is:```x --> [f1] --> a --> [f2] --> b --> [f3] --> y```**No checkpointing**: autograd saves `a` and `b` so backward can compute gradients.**Checkpoint after `a`**: you keep `a`, but you don't keep `b`. During backward, when you need `b` to differentiate `f3`, PyTorch reruns `f2(a)` to get it back.Memory goes down (you didn't store `b`). Compute goes up (you recomputed `b`).---## 0. Setup

In [75]:
# 0.1 Environment Check

import torch
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

PyTorch Version: 2.9.0+cu126
CUDA Available: True
CUDA Version: 12.6
GPU: Tesla T4
GPU Memory: 15.83 GB


---# 1. The Memory ProblemTraining deep neural networks hits a wall. That wall is GPU memory.Four things consume GPU memory during training:1. **Model parameters** (weights and biases)2. **Gradients** (same size as parameters)3. **Optimizer states** (for Adam: 2x parameter size for m and v buffers)4. **Activations** (intermediate outputs from each layer)For Adam/AdamW, the first three combined are 4x the model size (params + grads + 2 momentum buffers). This is fixed cost.Here is the surprise: **activations often dominate anyway**. Not parameters. Not gradients. Not optimizer states. Activations. Why? Because activations scale with batch size. Everything else does not.

In [76]:
# 1.1 Memory Breakdown: Theory, Measurement, and Timing

import torch
import torch.nn as nn

# ===== PART 1: Theoretical Breakdown =====

def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

def bytes_to_mb(b):
    return b / (1024 * 1024)

class DeepNetwork(nn.Module):
    def __init__(self, input_size=1024, hidden_size=1024, num_layers=10):
        super().__init__()
        layers = []
        for i in range(num_layers):
            in_f = input_size if i == 0 else hidden_size
            layers.append(nn.Linear(in_f, hidden_size))
            layers.append(nn.ReLU())
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

model = DeepNetwork(input_size=1024, hidden_size=1024, num_layers=10)
num_params = count_parameters(model)

# Theoretical memory breakdown for a full training setup with Adam
param_mem = num_params * 4  # fp32 parameters
grad_mem = param_mem        # gradients
optimizer_mem = param_mem * 2  # Adam's m and v buffers

print("=" * 60)
print("PART 1: THEORETICAL MEMORY BREAKDOWN")
print("=" * 60)
print(f"Model: {num_params:,} parameters")
print()
print("Fixed costs for training with Adam:")
print(f"  Parameter memory:  {bytes_to_mb(param_mem):.2f} MB")
print(f"  Gradient memory:   {bytes_to_mb(grad_mem):.2f} MB")
print(f"  Optimizer memory:  {bytes_to_mb(optimizer_mem):.2f} MB (m + v)")
print(f"  Total fixed:       {bytes_to_mb(param_mem + grad_mem + optimizer_mem):.2f} MB")
print()
print("Activation memory by batch size (THEORETICAL):")
fixed_cost = param_mem + grad_mem + optimizer_mem
for batch_size in [32, 128, 512]:
    act_mem = batch_size * 1024 * 10 * 4
    total = fixed_cost + act_mem
    act_pct = (act_mem / total) * 100
    print(f"  Batch {batch_size:3d}: activations = {bytes_to_mb(act_mem):6.2f} MB ({act_pct:.1f}% of total)")

# ===== PART 2: Real GPU Measurement =====

print()
print("=" * 60)
print("PART 2: REAL GPU MEASUREMENT")
print("=" * 60)

if torch.cuda.is_available():
    # Wider network to make activations dominate
    class WideNetwork(nn.Module):
        def __init__(self, hidden_size=4096, num_layers=6):
            super().__init__()
            layers = []
            for i in range(num_layers):
                layers.append(nn.Linear(1024 if i == 0 else hidden_size, hidden_size))
                layers.append(nn.ReLU())
            self.layers = nn.Sequential(*layers)
        
        def forward(self, x):
            return self.layers(x)

    model = WideNetwork().cuda()
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print()
    
    for batch_size in [64, 256, 512]:
        torch.cuda.reset_peak_memory_stats()
        torch.cuda.empty_cache()
        
        x = torch.randn(batch_size, 1024).cuda()
        output = model(x)
        loss = output.sum()
        loss.backward()
        
        peak = torch.cuda.max_memory_allocated() / 1e6
        act_estimate = batch_size * 4096 * 6 * 4 / 1e6
        print(f"Batch {batch_size:3d}: Peak = {peak:.1f} MB  (est. activations: {act_estimate:.1f} MB)")
        
        model.zero_grad()
        del x, output, loss
    
    del model
    torch.cuda.empty_cache()
else:
    print("[Run on GPU to see measurements]")

# ===== PART 3: Memory Timeline =====

print()
print("=" * 60)
print("PART 3: MEMORY TIMELINE (When Does Memory Peak?)")
print("=" * 60)

class InstrumentedNetwork(nn.Module):
    def __init__(self, num_layers=5, hidden=4096):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.Linear(hidden, hidden) for _ in range(num_layers)
        ])
    
    def forward(self, x, track=False):
        memory_log = []
        for i, layer in enumerate(self.layers):
            x = torch.relu(layer(x))
            if track and torch.cuda.is_available():
                torch.cuda.synchronize()
                memory_log.append(torch.cuda.memory_allocated() / 1e6)
        return x, memory_log

if torch.cuda.is_available():
    model = InstrumentedNetwork(num_layers=8, hidden=4096).cuda()
    x = torch.randn(256, 4096).cuda()
    
    num_params = sum(p.numel() for p in model.parameters())
    print(f"Model: {num_params/1e6:.1f}M params")
    print()
    
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.empty_cache()
    mem_model_only = torch.cuda.memory_allocated() / 1e6
    
    output, fwd_mem = model(x, track=True)
    mem_after_forward = torch.cuda.memory_allocated() / 1e6
    
    loss = output.sum()
    loss.backward()
    peak_memory = torch.cuda.max_memory_allocated() / 1e6
    mem_after_backward = torch.cuda.memory_allocated() / 1e6
    
    model.zero_grad(set_to_none=True)
    mem_after_zero_grad = torch.cuda.memory_allocated() / 1e6
    
    print("Memory Timeline:")
    print("-" * 55)
    print(f"{'Model loaded:':<35} {mem_model_only:>8.1f} MB")
    print()
    print("During forward (activations accumulating):")
    for i, mem in enumerate(fwd_mem):
        delta = mem - (fwd_mem[i-1] if i > 0 else mem_model_only)
        print(f"  After layer {i+1}:{'':<23} {mem:>8.1f} MB  (+{delta:.1f})")
    print()
    print(f"{'After forward (all activations live):':<35} {mem_after_forward:>8.1f} MB")
    print(f"{'PEAK (activations + gradients):':<35} {peak_memory:>8.1f} MB  <-- this kills you")
    print(f"{'After backward:':<35} {mem_after_backward:>8.1f} MB")
    print(f"{'After zero_grad:':<35} {mem_after_zero_grad:>8.1f} MB")
    print()
    
    activation_mem = mem_after_forward - mem_model_only
    print(f"KEY INSIGHT: Activation memory ~{activation_mem:.1f} MB dominates during forward pass")
    print("Memory peaks at start of backward when both activations and gradients exist")
else:
    print("[Run on GPU to see memory timeline]")

PART 1: THEORETICAL MEMORY BREAKDOWN
Model: 10,496,000 parameters

Fixed costs for training with Adam:
  Parameter memory:  40.04 MB
  Gradient memory:   40.04 MB
  Optimizer memory:  80.08 MB (m + v)
  Total fixed:       160.16 MB

Activation memory by batch size (THEORETICAL):
  Batch  32: activations =   1.25 MB (0.8% of total)
  Batch 128: activations =   5.00 MB (3.0% of total)
  Batch 512: activations =  20.00 MB (11.1% of total)

PART 2: REAL GPU MEASUREMENT
GPU: Tesla T4

Batch  64: Peak = 1983.6 MB  (est. activations: 6.3 MB)
Batch 256: Peak = 1990.6 MB  (est. activations: 25.2 MB)
Batch 512: Peak = 2000.1 MB  (est. activations: 50.3 MB)

PART 3: MEMORY TIMELINE (When Does Memory Peak?)
Model: 134.3M params

Memory Timeline:
-------------------------------------------------------
Model loaded:                         1817.6 MB

During forward (activations accumulating):
  After layer 1:                          1821.7 MB  (+4.2)
  After layer 2:                          1825.9

---# 2. Understanding Activation CheckpointingActivation checkpointing is a memory-compute trade-off. Save memory by not storing activations. Pay for it by recomputing them during backprop.To understand this, you need to know what activations are and why backprop needs them.

In [77]:
# 2.1 Why Backprop Needs Activations

import torch

# y = W2 * relu(W1 * x)
# Gradient w.r.t. W1 needs the pre-activation W1*x to compute relu'(W1*x)

torch.manual_seed(42)

W1 = torch.randn(4, 4, requires_grad=True)
W2 = torch.randn(4, 4, requires_grad=True)
x = torch.randn(1, 4)

# Forward pass stores these
z1 = x @ W1.T           # Pre-activation (needed for ReLU grad)
a1 = torch.relu(z1)     # Activation (needed for W2 grad)
y = a1 @ W2.T

print("Stored for backprop:")
print(f"  z1: {z1.shape} - needed to compute relu gradient")
print(f"  a1: {a1.shape} - needed to compute W2 gradient")

loss = y.sum()
loss.backward()

print(f"\nGradients computed:")
print(f"  W1.grad: {W1.grad.shape}")
print(f"  W2.grad: {W2.grad.shape}")
print()
print("The chain rule requires intermediate values.")
print("PyTorch keeps them alive until backward finishes.")
print("This is why activations accumulate through forward pass.")

Stored for backprop:
  z1: torch.Size([1, 4]) - needed to compute relu gradient
  a1: torch.Size([1, 4]) - needed to compute W2 gradient

Gradients computed:
  W1.grad: torch.Size([4, 4])
  W2.grad: torch.Size([4, 4])

The chain rule requires intermediate values.
PyTorch keeps them alive until backward finishes.
This is why activations accumulate through forward pass.


In [78]:
# 2.2 The Core Trade-off: Store vs Recompute

import math

print("Standard Training:")
print("  Forward: x -> a1 -> a2 -> a3 -> loss")
print("           [store] [store] [store]")
print("  Backward: uses stored a1, a2, a3")
print("  Memory: O(n) activations")
print()
print("With Checkpointing:")
print("  Forward: x -> a1 -> a2 -> a3 -> loss")
print("           [save]       [save]")
print("  Backward: recompute a2 from a1, then use")
print("  Memory: O(sqrt(n)) with optimal placement")
print()

num_layers = 100
optimal = int(math.sqrt(num_layers))

print(f"For {num_layers} layers:")
print(f"  Standard: {num_layers} activations stored")
print(f"  Checkpointed: ~{optimal} activations stored (at checkpoint boundaries)")
print(f"  Memory reduction: {num_layers/optimal:.0f}x")
print()
print("Compute analysis:")
print(f"  Without checkpointing: n forward + n backward = 2n")
print(f"  With checkpointing:    n forward + n backward + n recompute = 3n")
print(f"  Compute overhead: ~50% of total training time")
print()
print("KEY: The sqrt(n) rule - optimal checkpointing reduces memory from O(n) to O(sqrt(n))")

Standard Training:
  Forward: x -> a1 -> a2 -> a3 -> loss
           [store] [store] [store]
  Backward: uses stored a1, a2, a3
  Memory: O(n) activations

With Checkpointing:
  Forward: x -> a1 -> a2 -> a3 -> loss
           [save]       [save]
  Backward: recompute a2 from a1, then use
  Memory: O(sqrt(n)) with optimal placement

For 100 layers:
  Standard: 100 activations stored
  Checkpointed: ~10 activations stored (at checkpoint boundaries)
  Memory reduction: 10x

Compute analysis:
  Without checkpointing: n forward + n backward = 2n
  With checkpointing:    n forward + n backward + n recompute = 3n
  Compute overhead: ~50% of total training time

KEY: The sqrt(n) rule - optimal checkpointing reduces memory from O(n) to O(sqrt(n))


---# 3. PyTorch API - Basic UsagePyTorch provides `torch.utils.checkpoint.checkpoint()` for activation checkpointing. The basic pattern is simple: wrap the function you want to checkpoint.

In [79]:
# 3.1 Basic Checkpoint: Side-by-Side Comparison

import torch
import torch.nn as nn
from torch.utils.checkpoint import checkpoint

# ===== WITHOUT Checkpointing =====
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(1024, 1024)
        self.layer2 = nn.Linear(1024, 1024)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        return x

# ===== WITH Checkpointing =====
class CheckpointedModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(1024, 1024)
        self.layer2 = nn.Linear(1024, 1024)

    def forward(self, x):
        # Run layer1 normally (so input to checkpointed region requires grad)
        x = torch.relu(self.layer1(x))
        # Checkpoint layer2: activations won't be stored, will be recomputed
        x = checkpoint(lambda t: torch.relu(self.layer2(t)), x, use_reentrant=False)
        return x

if torch.cuda.is_available():
    # Test without checkpointing
    model1 = SimpleModel().cuda()
    x1 = torch.randn(1, 1024).cuda()
    torch.cuda.reset_peak_memory_stats()
    out1 = model1(x1)
    out1.sum().backward()
    mem_no_ckpt = torch.cuda.max_memory_allocated() / 1e6
    del model1, x1, out1
    torch.cuda.empty_cache()
    
    # Test with checkpointing
    model2 = CheckpointedModel().cuda()
    x2 = torch.randn(1, 1024).cuda()
    torch.cuda.reset_peak_memory_stats()
    out2 = model2(x2)
    out2.sum().backward()
    mem_with_ckpt = torch.cuda.max_memory_allocated() / 1e6
    
    print("Basic Checkpoint Comparison (small model)")
    print("-" * 50)
    print(f"WITHOUT Checkpointing: {mem_no_ckpt:.2f} MB")
    print(f"WITH Checkpointing:    {mem_with_ckpt:.2f} MB")
    print()
    print("NOTE: Small model shows minimal difference.")
    print("Savings become significant with deeper models (see Section 4).")
else:
    print("[Run on GPU to see memory comparison]")
    print()
    print("API Usage:")
    print("  checkpoint(fn, *args, use_reentrant=False)")
    print("  - fn: callable to checkpoint")
    print("  - Always use use_reentrant=False (modern API)")

Basic Checkpoint Comparison (small model)
--------------------------------------------------
WITHOUT Checkpointing: 1834.36 MB
WITH Checkpointing:    1815.92 MB

NOTE: Small model shows minimal difference.
Savings become significant with deeper models (see Section 4).


---# 4. PyTorch API - Sequential ModelsFor deep sequential models, use `checkpoint_sequential`. It splits the model into segments and checkpoints each segment.

In [ ]:
# 4.1 Deep Model Comparison: The Real Difference

import gc
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.checkpoint import checkpoint

class FFNBlock(nn.Module):
    """FFN block with 4x expansion - the memory-hungry part of transformers."""
    def __init__(self, dim):
        super().__init__()
        self.fc1 = nn.Linear(dim, dim * 4)
        self.fc2 = nn.Linear(dim * 4, dim)
    
    def forward(self, x):
        return self.fc2(F.gelu(self.fc1(x)))

if torch.cuda.is_available():
    num_layers = 12
    hidden = 2048
    batch_size = 128
    
    # ===== Test 1: WITHOUT checkpointing =====
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    
    model1 = nn.ModuleList([FFNBlock(hidden) for _ in range(num_layers)]).cuda()
    x1 = torch.randn(batch_size, hidden, device='cuda', requires_grad=True)
    
    out = x1
    for block in model1:
        out = block(out)
    out.sum().backward()
    
    mem_no_ckpt = torch.cuda.max_memory_allocated() / 1e6
    
    del model1, x1, out
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    
    # ===== Test 2: WITH checkpointing =====
    model2 = nn.ModuleList([FFNBlock(hidden) for _ in range(num_layers)]).cuda()
    x2 = torch.randn(batch_size, hidden, device='cuda', requires_grad=True)
    
    out2 = x2
    for block in model2:
        out2 = checkpoint(block, out2, use_reentrant=False)
    out2.sum().backward()
    
    mem_ckpt = torch.cuda.max_memory_allocated() / 1e6
    
    saved = mem_no_ckpt - mem_ckpt
    pct = (saved / mem_no_ckpt) * 100 if mem_no_ckpt > 0 else 0
    
    print(f"Config: {num_layers} FFN blocks (4x expansion), hidden={hidden}, batch={batch_size}")
    print("=" * 60)
    print()
    print(f"Peak memory WITHOUT checkpointing: {mem_no_ckpt:,.0f} MB")
    print(f"Peak memory WITH checkpointing:    {mem_ckpt:,.0f} MB")
    print(f"Memory saved:                      {saved:,.0f} MB ({pct:.1f}%)")
    print()
    print("WHY THIS MATTERS:")
    print(f"  - You could increase batch size by ~{int(saved/100)*10}% before hitting OOM")
    print(f"  - Or fit ~{pct:.0f}% more layers in the same memory")
    print()
    print("TRADE-OFF: ~50% more compute (one extra forward pass per block)")
else:
    print("[Run on GPU]")

In [102]:
# 4.2 Using checkpoint_sequential

import torch
import torch.nn as nn
from torch.utils.checkpoint import checkpoint_sequential

# A deep sequential model
model = nn.Sequential(
    nn.Linear(1024, 1024), nn.ReLU(),
    nn.Linear(1024, 1024), nn.ReLU(),
    nn.Linear(1024, 1024), nn.ReLU(),
    nn.Linear(1024, 1024), nn.ReLU(),
)

x = torch.randn(1, 1024)
if torch.cuda.is_available():
    model = model.cuda()
    x = x.cuda()

# Split into 2 checkpoint segments
segments = 2
output = checkpoint_sequential(model, segments, x, use_reentrant=False)

print(f"Output shape: {output.shape}")
print(f"Model split into {segments} checkpoint segments")
print("Each segment recomputes activations during backward pass")
print()
print("API: checkpoint_sequential(sequential_model, segments, input, use_reentrant=False)")
print("- Convenience wrapper for nn.Sequential models")
print("- Works well with FSDP and DDP")

Output shape: torch.Size([1, 1024])
Model split into 2 checkpoint segments
Each segment recomputes activations during backward pass

API: checkpoint_sequential(sequential_model, segments, input, use_reentrant=False)
- Convenience wrapper for nn.Sequential models
- Works well with FSDP and DDP


---# 5. Practical Pattern - TransformersIn transformers, the feed-forward block uses the most memory (4x expansion). This is where checkpointing helps most.

In [82]:
# 5.1 Transformer Training with Checkpointing

import torch
import torch.nn as nn
from torch.utils.checkpoint import checkpoint

class TransformerBlock(nn.Module):
    def __init__(self, embed_size, num_heads=8):
        super().__init__()
        self.attention = nn.MultiheadAttention(embed_size, num_heads, batch_first=True)
        self.norm1 = nn.LayerNorm(embed_size)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, embed_size * 4),
            nn.GELU(),
            nn.Linear(embed_size * 4, embed_size)
        )
        self.norm2 = nn.LayerNorm(embed_size)

    def forward(self, x):
        # Self-attention with residual
        attn_out, _ = self.attention(x, x, x)
        x = self.norm1(x + attn_out)
        # Feed-forward with residual (checkpointed)
        ff_out = checkpoint(self.feed_forward, x, use_reentrant=False)
        x = self.norm2(x + ff_out)
        return x

# Example usage
embed_size = 512
seq_length = 64
batch_size = 8

block = TransformerBlock(embed_size)
if torch.cuda.is_available():
    block = block.cuda()

x = torch.randn(batch_size, seq_length, embed_size)
if torch.cuda.is_available():
    x = x.cuda()

output = block(x)
print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")
print()
print("Feed-forward block is checkpointed (it uses the most memory)")
print("Attention is NOT checkpointed (expensive to recompute)")
print()
print("Pattern: Checkpoint the FFN, keep attention cached")
print("This is how most transformer implementations do it.")

Input shape: torch.Size([8, 64, 512])
Output shape: torch.Size([8, 64, 512])

Feed-forward block is checkpointed (it uses the most memory)
Attention is NOT checkpointed (expensive to recompute)

Pattern: Checkpoint the FFN, keep attention cached
This is how most transformer implementations do it.


---# 6. Debugging and Best PracticesCheckpointing has gotchas. Here are the common ones and how to avoid them.

In [83]:
# 6.1 Pitfall: Non-Differentiable Operations

import torch
from torch.utils.checkpoint import checkpoint

# BAD: torch.no_grad() inside checkpointed function
def bad_forward(x):
    with torch.no_grad():  # This breaks gradient computation!
        x = x ** 2
    return x

# GOOD: Keep everything differentiable
def good_forward(x):
    x = x ** 2  # No torch.no_grad()
    return x

x = torch.randn(4, requires_grad=True)

# This will fail or give wrong gradients
try:
    y_bad = checkpoint(bad_forward, x, use_reentrant=False)
    y_bad.sum().backward()
    print("BAD: Gradients might be zero or wrong")
except Exception as e:
    print(f"BAD: Error - {e}")

# This works
x = torch.randn(4, requires_grad=True)
y_good = checkpoint(good_forward, x, use_reentrant=False)
y_good.sum().backward()
print(f"GOOD: x.grad = {x.grad}")
print()
print("RULE: Everything inside a checkpointed function must be differentiable.")
print("No torch.no_grad(), no tensor.detach(), no in-place ops that break autograd.")

BAD: Error - element 0 of tensors does not require grad and does not have a grad_fn
GOOD: x.grad = tensor([-0.0629,  4.8470,  1.8998, -1.8578])

RULE: Everything inside a checkpointed function must be differentiable.
No torch.no_grad(), no tensor.detach(), no in-place ops that break autograd.


In [84]:
# 6.2 Pitfall: Random Operations (Dropout)

import torch
import torch.nn as nn
from torch.utils.checkpoint import checkpoint

# Problem: Dropout uses random masks.
# If RNG state is NOT preserved, the recomputed forward in backward
# will sample a different mask than the original forward.
class ModelWithDropout(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(32, 32)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.linear(x)
        x = self.dropout(x)  # Random! Different each forward pass
        return x

model = ModelWithDropout()
x = torch.randn(4, 32, requires_grad=True)

# Fix: preserve RNG state (default) so recomputation matches the original forward.
y = checkpoint(model, x, use_reentrant=False, preserve_rng_state=True)
print(f"Output shape: {y.shape}")
print("RNG state preserved: dropout mask matches between forward and recompute")
print()
print("KEY: preserve_rng_state=True is the default for use_reentrant=False")
print("Dropout and other random ops work correctly without extra configuration.")

Output shape: torch.Size([4, 32])
RNG state preserved: dropout mask matches between forward and recompute

KEY: preserve_rng_state=True is the default for use_reentrant=False
Dropout and other random ops work correctly without extra configuration.


In [85]:
# 6.3 Best Practices Summary

print("""
BEST PRACTICES FOR ACTIVATION CHECKPOINTING
============================================

1. USE use_reentrant=False
   - Modern API, handles edge cases better
   - checkpoint(fn, x, use_reentrant=False)

2. CHECKPOINT LARGE BLOCKS, NOT SMALL LAYERS
   - Overhead of checkpoint() call is non-trivial
   - Group 2-4 layers into blocks, then checkpoint blocks

3. AVOID NESTED CHECKPOINTS
   - Don't checkpoint inside checkpointed functions
   - Leads to exponential recomputation

4. KEEP EVERYTHING DIFFERENTIABLE
   - No torch.no_grad() inside checkpointed regions
   - No tensor detaching
   - No in-place ops that break autograd

5. TEST GRADIENTS FIRST
   - Compare gradients with and without checkpointing
   - They should be identical (within float precision)

6. PROFILE MEMORY
   - Use torch.cuda.memory_stats() to verify savings
   - Peak memory is what matters

7. CONSIDER CHECKPOINT PLACEMENT
   - Middle layers often have largest activations
   - Checkpoint those first
""")

# Gradient verification example
import torch
import torch.nn as nn
from torch.utils.checkpoint import checkpoint

model = nn.Linear(64, 64)
x = torch.randn(8, 64, requires_grad=True)

# Without checkpoint
x1 = x.clone().detach().requires_grad_(True)
y1 = model(x1)
y1.sum().backward()
grad1 = x1.grad.clone()

# With checkpoint
x2 = x.clone().detach().requires_grad_(True)
y2 = checkpoint(model, x2, use_reentrant=False)
y2.sum().backward()
grad2 = x2.grad.clone()

print(f"Gradients match: {torch.allclose(grad1, grad2)}")
print(f"Max difference: {(grad1 - grad2).abs().max():.2e}")
print()
print("ALWAYS verify gradients when adding checkpointing to new code.")


BEST PRACTICES FOR ACTIVATION CHECKPOINTING

1. USE use_reentrant=False
   - Modern API, handles edge cases better
   - checkpoint(fn, x, use_reentrant=False)

2. CHECKPOINT LARGE BLOCKS, NOT SMALL LAYERS
   - Overhead of checkpoint() call is non-trivial
   - Group 2-4 layers into blocks, then checkpoint blocks

3. AVOID NESTED CHECKPOINTS
   - Don't checkpoint inside checkpointed functions
   - Leads to exponential recomputation

4. KEEP EVERYTHING DIFFERENTIABLE
   - No torch.no_grad() inside checkpointed regions
   - No tensor detaching
   - No in-place ops that break autograd

5. TEST GRADIENTS FIRST
   - Compare gradients with and without checkpointing
   - They should be identical (within float precision)

6. PROFILE MEMORY
   - Use torch.cuda.memory_stats() to verify savings
   - Peak memory is what matters

7. CONSIDER CHECKPOINT PLACEMENT
   - Middle layers often have largest activations
   - Checkpoint those first

Gradients match: True
Max difference: 0.00e+00

ALWAYS verif

---# 7. Selective Activation Checkpoint (SAC)Standard checkpointing is all-or-nothing. Every op in the checkpointed region gets recomputed during backward.Selective Activation Checkpoint (SAC) gives you granular control. You choose which operations to save and which to recompute.Why does this matter? Not all operations are equal:- Matmuls are expensive to recompute- Pointwise ops (relu, sigmoid) are cheap- Attention is very expensiveSAC lets you save the expensive ones and recompute the cheap ones. Best of both worlds.

In [86]:
# 7.1 The CheckpointPolicy Enum

try:
    from torch.utils.checkpoint import CheckpointPolicy
except Exception as e:
    CheckpointPolicy = None
    print("CheckpointPolicy not available in this PyTorch build.")
    print(f"Details: {type(e).__name__}: {e}")

if CheckpointPolicy is not None:
    print("CheckpointPolicy has four options:")
    print("-" * 50)
    print()
    print("MUST_SAVE:")
    print("  - Always save this op's output")
    print("  - Never recompute it")
    print("  - Use for expensive ops (matmul, attention)")
    print()
    print("PREFER_SAVE:")
    print("  - Save if possible")
    print("  - torch.compile may override this")
    print()
    print("MUST_RECOMPUTE:")
    print("  - Always recompute this op")
    print("  - Never save its output")
    print()
    print("PREFER_RECOMPUTE:")
    print("  - Recompute if possible")
    print("  - torch.compile may override this")
    print("  - Use for cheap ops (relu, elementwise)")
    print()
    print("The MUST_ variants are strict. The PREFER_ variants are hints.")
    print()
    print("RULE: MUST_SAVE for expensive ops, PREFER_RECOMPUTE for cheap ops.")

CheckpointPolicy has four options:
--------------------------------------------------

MUST_SAVE:
  - Always save this op's output
  - Never recompute it
  - Use for expensive ops (matmul, attention)

PREFER_SAVE:
  - Save if possible
  - torch.compile may override this

MUST_RECOMPUTE:
  - Always recompute this op
  - Never save its output

PREFER_RECOMPUTE:
  - Recompute if possible
  - torch.compile may override this
  - Use for cheap ops (relu, elementwise)

The MUST_ variants are strict. The PREFER_ variants are hints.

RULE: MUST_SAVE for expensive ops, PREFER_RECOMPUTE for cheap ops.


In [87]:
# 7.2 Policy Functions: What to Save vs Recompute

import torch

try:
    from torch.utils.checkpoint import CheckpointPolicy
except Exception as e:
    CheckpointPolicy = None
    print("CheckpointPolicy not available; skipping SAC policy definition.")
    print(f"Details: {type(e).__name__}: {e}")

aten = torch.ops.aten

def _maybe_default(op_name: str):
    """Return torch.ops.aten.<op_name>.default if it exists, else None."""
    try:
        return getattr(getattr(aten, op_name), "default")
    except AttributeError:
        return None

# ===== Policy 1: Conservative - Save only matmuls =====
compute_intensive_ops_basic = [
    _maybe_default("mm"),
    _maybe_default("bmm"),
    _maybe_default("addmm"),
]
compute_intensive_ops_basic = [op for op in compute_intensive_ops_basic if op is not None]

def policy_save_matmuls(ctx, op, *args, **kwargs):
    """Save matmuls, recompute everything else."""
    if CheckpointPolicy is None:
        raise RuntimeError("CheckpointPolicy is not available in this PyTorch build.")
    if op in compute_intensive_ops_basic:
        return CheckpointPolicy.MUST_SAVE
    return CheckpointPolicy.PREFER_RECOMPUTE

# ===== Policy 2: Aggressive - Save all expensive ops =====
op_names = [
    "mm", "bmm", "addmm", "convolution", "upsample_bilinear2d",
    "_scaled_mm", "linear",
    "_scaled_dot_product_flash_attention",
    "_scaled_dot_product_efficient_attention",
]

compute_intensive_ops_full = []
missing = []
for name in op_names:
    op = _maybe_default(name)
    if op is None:
        missing.append(name)
    else:
        compute_intensive_ops_full.append(op)

def policy_save_all_expensive(ctx, op, *args, **kwargs):
    """Save all compute-intensive ops, including attention (when present)."""
    if CheckpointPolicy is None:
        raise RuntimeError("CheckpointPolicy is not available in this PyTorch build.")
    if op in compute_intensive_ops_full:
        return CheckpointPolicy.MUST_SAVE
    return CheckpointPolicy.PREFER_RECOMPUTE

print("Two SAC Policies Defined:")
print("=" * 60)
print()
print("Policy 1: policy_save_matmuls (Conservative)")
print("  Saves: mm, bmm, addmm")
print("  Recomputes: relu, gelu, sigmoid, layernorm, etc.")
print("  Use when: Speed is important, memory savings secondary")
print()
print("Policy 2: policy_save_all_expensive (Aggressive)")
print("  Saves: matmuls + convolutions + attention + upsampling")
print("  Recomputes: only cheap pointwise ops")
if missing:
    print(f"  (Missing ops in this build: {', '.join(missing)})")
print("  Use when: Maximum memory savings needed")
print()
print("NOTE: Always match against .default (e.g., aten.mm.default)")
print("to match the actual ops passed to the policy function.")

Two SAC Policies Defined:

Policy 1: policy_save_matmuls (Conservative)
  Saves: mm, bmm, addmm
  Recomputes: relu, gelu, sigmoid, layernorm, etc.
  Use when: Speed is important, memory savings secondary

Policy 2: policy_save_all_expensive (Aggressive)
  Saves: matmuls + convolutions + attention + upsampling
  Recomputes: only cheap pointwise ops
  Use when: Maximum memory savings needed

NOTE: Always match against .default (e.g., aten.mm.default)
to match the actual ops passed to the policy function.


In [88]:
# 7.3 Using SAC in Practice

import functools
import torch
from torch.utils.checkpoint import checkpoint

try:
    from torch.utils.checkpoint import CheckpointPolicy, create_selective_checkpoint_contexts
except Exception as e:
    CheckpointPolicy = None
    create_selective_checkpoint_contexts = None
    print("Selective checkpointing APIs not available in this PyTorch build.")
    print(f"Details: {type(e).__name__}: {e}")

if CheckpointPolicy is not None and create_selective_checkpoint_contexts is not None:
    aten = torch.ops.aten

    def _maybe_default(op_name: str):
        try:
            return getattr(getattr(aten, op_name), "default")
        except AttributeError:
            return None

    # Build op allowlist dynamically
    ops_to_save = [
        _maybe_default("mm"),
        _maybe_default("bmm"),
        _maybe_default("addmm"),
    ]
    ops_to_save = [op for op in ops_to_save if op is not None]

    # ===== Method 1: Explicit policy function =====
    def policy_fn(ctx, op, *args, **kwargs):
        if op in ops_to_save:
            return CheckpointPolicy.MUST_SAVE
        return CheckpointPolicy.PREFER_RECOMPUTE

    # ===== Method 2: Allowlist shortcut =====
    def allowlist_policy(ops):
        ops = set(ops)
        def _policy(ctx, op, *args, **kwargs):
            if op in ops:
                return CheckpointPolicy.MUST_SAVE
            return CheckpointPolicy.PREFER_RECOMPUTE
        return _policy

    # Create the context function
    context_fn = functools.partial(create_selective_checkpoint_contexts, policy_fn)

    def _run_checkpoint(fn, *args, context_fn=None):
        """Call checkpoint() with best-effort compatibility across versions."""
        kwargs = {"use_reentrant": False}
        if context_fn is not None:
            kwargs["context_fn"] = context_fn
        try:
            return checkpoint(fn, *args, **kwargs)
        except TypeError:
            kwargs.pop("context_fn", None)
            try:
                return checkpoint(fn, *args, **kwargs)
            except TypeError:
                kwargs.pop("use_reentrant", None)
                return checkpoint(fn, *args)

    def forward_fn(x, weight1, weight2):
        x = torch.mm(x, weight1)   # Saved (matmul)
        x = torch.relu(x)          # Recomputed (cheap)
        x = torch.mm(x, weight2)   # Saved (matmul)
        x = torch.sigmoid(x)       # Recomputed (cheap)
        return x

    x = torch.randn(32, 64, requires_grad=True)
    w1 = torch.randn(64, 64, requires_grad=True)
    w2 = torch.randn(64, 64, requires_grad=True)

    output = _run_checkpoint(forward_fn, x, w1, w2, context_fn=context_fn)

    print("SAC API Usage:")
    print("-" * 50)
    print("1. Define policy_fn(ctx, op, *args, **kwargs) -> CheckpointPolicy")
    print("2. Create context_fn with functools.partial")
    print("3. Pass context_fn to checkpoint(..., context_fn=context_fn)")
    print()
    print(f"Output shape: {output.shape}")
    print("Matmuls saved. Cheap activations will be recomputed during backward.")
    print()
    print("SHORTCUT: Use allowlist_policy(ops) to avoid boilerplate.")
else:
    print("[Skipping SAC demo: missing CheckpointPolicy/create_selective_checkpoint_contexts]")

SAC API Usage:
--------------------------------------------------
1. Define policy_fn(ctx, op, *args, **kwargs) -> CheckpointPolicy
2. Create context_fn with functools.partial
3. Pass context_fn to checkpoint(..., context_fn=context_fn)

Output shape: torch.Size([32, 64])
Matmuls saved. Cheap activations will be recomputed during backward.

SHORTCUT: Use allowlist_policy(ops) to avoid boilerplate.


In [89]:
# 7.4 SAC vs Standard AC: Memory and Time Comparison

import functools
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.checkpoint import checkpoint

try:
    from torch.utils.checkpoint import CheckpointPolicy, create_selective_checkpoint_contexts
except Exception as e:
    CheckpointPolicy = None
    create_selective_checkpoint_contexts = None
    print("Selective checkpointing APIs not available in this PyTorch build.")
    print(f"Details: {type(e).__name__}: {e}")

class TransformerFFN(nn.Module):
    """Feed-forward block from a transformer."""
    def __init__(self, dim=1024, expansion=4):
        super().__init__()
        self.fc1 = nn.Linear(dim, dim * expansion)
        self.fc2 = nn.Linear(dim * expansion, dim)

    def forward(self, x):
        x = self.fc1(x)
        x = F.gelu(x)  # Cheap pointwise
        x = self.fc2(x)
        return x

def _run_checkpoint(fn, *args, context_fn=None):
    """Call checkpoint() with best-effort compatibility across versions."""
    kwargs = {"use_reentrant": False}
    if context_fn is not None:
        kwargs["context_fn"] = context_fn
    try:
        return checkpoint(fn, *args, **kwargs)
    except TypeError:
        kwargs.pop("context_fn", None)
        try:
            return checkpoint(fn, *args, **kwargs)
        except TypeError:
            return checkpoint(fn, *args)

if torch.cuda.is_available():
    dim = 2048
    batch = 64
    seq = 128

    # Add a small stem so the checkpointed region receives activations that require grad.
    stem = nn.Linear(dim, dim).cuda()
    ffn = TransformerFFN(dim).cuda()

    x = torch.randn(batch, seq, dim, device="cuda")

    sac_context = None
    if CheckpointPolicy is not None and create_selective_checkpoint_contexts is not None:
        aten = torch.ops.aten

        def _maybe_default(op_name: str):
            try:
                return getattr(getattr(aten, op_name), "default")
            except AttributeError:
                return None

        saved_ops = [op for op in [_maybe_default("mm"), _maybe_default("addmm")] if op is not None]

        def sac_policy(ctx, op, *args, **kwargs):
            # nn.Linear typically lowers to addmm/mm
            if op in saved_ops:
                return CheckpointPolicy.MUST_SAVE
            return CheckpointPolicy.PREFER_RECOMPUTE

        sac_context = functools.partial(create_selective_checkpoint_contexts, sac_policy)
    else:
        print("SAC APIs not available: will skip Selective AC benchmark")

    def bench(mode: str, iters: int = 5, warmup: int = 2):
        def step():
            h = stem(x)
            if mode == "none":
                out = ffn(h)
            elif mode == "standard":
                out = _run_checkpoint(ffn, h)
            elif mode == "selective":
                if sac_context is None:
                    raise RuntimeError("Selective checkpointing context not available")
                out = _run_checkpoint(ffn, h, context_fn=sac_context)
            else:
                raise ValueError(mode)

            out.sum().backward()
            stem.zero_grad(set_to_none=True)
            ffn.zero_grad(set_to_none=True)

        for _ in range(warmup):
            step()

        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()
        torch.cuda.synchronize()

        start = time.time()
        for _ in range(iters):
            step()

        torch.cuda.synchronize()
        time_ms = (time.time() - start) / iters * 1000
        mem_mb = torch.cuda.max_memory_allocated() / 1e6
        return mem_mb, time_ms

    results = {
        "No Checkpoint": bench("none"),
        "Standard AC": bench("standard"),
    }
    if sac_context is not None:
        results["Selective AC"] = bench("selective")

    print(f"Transformer FFN: dim={dim}, batch={batch}, seq={seq}")
    print("-" * 70)
    print(f"{'Mode':<15} {'Peak Memory (MB)':<18} {'Time (ms/iter)':<15}")
    print("-" * 70)
    for name, (mem, t) in results.items():
        print(f"{name:<15} {mem:<18.2f} {t:<15.2f}")
    print("-" * 70)
    print("Standard AC: recomputes everything inside the checkpointed region")
    print("Selective AC (when available): can save matmuls and recompute only GELU")
else:
    print("[Run on GPU for comparison]")

Transformer FFN: dim=2048, batch=64, seq=128
----------------------------------------------------------------------
Mode            Peak Memory (MB)   Time (ms/iter) 
----------------------------------------------------------------------
No Checkpoint   2567.75            448.69         
Standard AC     2567.75            528.41         
Selective AC    2567.75            462.01         
----------------------------------------------------------------------
Standard AC: recomputes everything inside the checkpointed region
Selective AC (when available): can save matmuls and recompute only GELU


---# 8. Distributed Training and Mixed PrecisionCheckpointing works seamlessly with DDP and mixed precision. No special configuration needed.

In [90]:
# 8.1 DDP + Checkpointing (Code Template)
# NOTE: This code shows the pattern. Run in a distributed environment.

"""
import torch
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.checkpoint import checkpoint

# Initialize distributed
dist.init_process_group(backend="nccl")
local_rank = int(os.environ["LOCAL_RANK"])
torch.cuda.set_device(local_rank)

class ModelWithCheckpoint(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(1024, 1024)
        self.layer2 = nn.Linear(1024, 1024)
    
    def forward(self, x):
        x = checkpoint(lambda t: torch.relu(self.layer1(t)), x, use_reentrant=False)
        x = torch.relu(self.layer2(x))
        return x

# Wrap with DDP
model = ModelWithCheckpoint().cuda(local_rank)
ddp_model = DDP(model, device_ids=[local_rank])

# Training works as normal
x = torch.randn(16, 1024).cuda(local_rank)
output = ddp_model(x)
loss = output.sum()
loss.backward()
"""

print("DDP + Checkpointing:")
print("- Checkpointing happens locally on each GPU")
print("- DDP handles gradient synchronization")
print("- No special configuration needed")
print("- Memory savings apply per-GPU")

DDP + Checkpointing:
- Checkpointing happens locally on each GPU
- DDP handles gradient synchronization
- No special configuration needed
- Memory savings apply per-GPU


In [91]:
# 8.2 Mixed Precision + Checkpointing

import torch
import torch.nn as nn
from torch.utils.checkpoint import checkpoint

class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(1024, 1024)
        self.layer2 = nn.Linear(1024, 1024)
    
    def forward(self, x):
        x = checkpoint(lambda t: torch.relu(self.layer1(t)), x, use_reentrant=False)
        x = torch.relu(self.layer2(x))
        return x

if torch.cuda.is_available():
    model = SimpleModel().cuda()
    optimizer = torch.optim.Adam(model.parameters())
    scaler = torch.cuda.amp.GradScaler()
    
    # Training loop with mixed precision + checkpointing
    for step in range(3):
        x = torch.randn(16, 1024).cuda()
        
        with torch.cuda.amp.autocast():
            output = model(x)
            loss = output.sum()
        
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        print(f"Step {step}: loss = {loss.item():.4f}")
    
    print("\nMixed precision + checkpointing works seamlessly")
else:
    print("[Run on GPU]")

Step 0: loss = 1532.7600
Step 1: loss = 1574.5704
Step 2: loss = 1518.4087

Mixed precision + checkpointing works seamlessly


/tmp/ipython-input-237665819.py:21: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
/tmp/ipython-input-237665819.py:27: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


---# 9. torch.compile and Memory Budget APItorch.compile (PyTorch 2.0+) traces your forward and backward passes into a single joint graph and applies a "min-cut" partitioner that automatically decides which tensors to save and which to recompute.By default, min-cut prioritizes speed, not memory. The Memory Budget API gives you control over this trade-off.

In [92]:
# 9.1 torch.compile: The Min-Cut Partitioner

import torch
import torch.nn as nn

print("How torch.compile handles activations:")
print("-" * 50)
print()
print("1. TRACING")
print("   torch.compile traces forward AND backward into one graph.")
print("   This lets it see the whole picture.")
print()
print("2. MIN-CUT PARTITIONING")
print("   The graph is split at the optimal points.")
print("   Algorithm minimizes: tensors crossing the cut")
print("   (These are the tensors saved for backward)")
print()
print("3. AUTOMATIC RECOMPUTATION")
print("   Cheap ops (relu, add, mul) are recomputed automatically.")
print("   No user intervention needed.")
print()
print("4. FUSION")
print("   Pointwise ops get fused into kernels.")
print("   Fused ops are fast to recompute.")
print()
print("Result: torch.compile gives you SOME memory savings for FREE,")
print("        plus speed improvements from fusion.")

How torch.compile handles activations:
--------------------------------------------------

1. TRACING
   torch.compile traces forward AND backward into one graph.
   This lets it see the whole picture.

2. MIN-CUT PARTITIONING
   The graph is split at the optimal points.
   Algorithm minimizes: tensors crossing the cut
   (These are the tensors saved for backward)

3. AUTOMATIC RECOMPUTATION
   Cheap ops (relu, add, mul) are recomputed automatically.
   No user intervention needed.

4. FUSION
   Pointwise ops get fused into kernels.
   Fused ops are fast to recompute.

Result: torch.compile gives you SOME memory savings for FREE,
        plus speed improvements from fusion.


In [93]:
# 9.2 Comparing: Eager vs Compile vs Checkpointing

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.checkpoint import checkpoint

class SimpleFFN(nn.Module):
    def __init__(self, dim=2048):
        super().__init__()
        self.fc1 = nn.Linear(dim, dim * 4)
        self.fc2 = nn.Linear(dim * 4, dim)
    
    def forward(self, x):
        x = self.fc1(x)
        x = F.gelu(x)
        x = self.fc2(x)
        return x

class Wrapper(nn.Module):
    """Adds a small stem so the checkpointed region receives activations that require grad."""
    def __init__(self, dim=2048, use_checkpoint=False):
        super().__init__()
        self.stem = nn.Linear(dim, dim)
        self.ffn = SimpleFFN(dim)
        self.use_checkpoint = use_checkpoint

    def forward(self, x):
        x = self.stem(x)
        if self.use_checkpoint:
            x = checkpoint(self.ffn, x, use_reentrant=False)
        else:
            x = self.ffn(x)
        return x

def warmup(fn, zero_grad_fn, x, iters=2):
    for _ in range(iters):
        fn(x).sum().backward()
        zero_grad_fn()

def peak_memory_mb(fn, zero_grad_fn, x):
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    out = fn(x)
    out.sum().backward()
    peak = torch.cuda.max_memory_allocated() / 1e6
    zero_grad_fn()
    return peak

if torch.cuda.is_available():
    dim = 2048
    batch = 64
    seq = 128

    x = torch.randn(batch, seq, dim, device="cuda")

    eager_model = Wrapper(dim, use_checkpoint=False).cuda()
    ckpt_model = Wrapper(dim, use_checkpoint=True).cuda()

    results = {}

    # 1) Eager mode
    warmup(eager_model, lambda: eager_model.zero_grad(set_to_none=True), x)
    results['Eager'] = peak_memory_mb(eager_model, lambda: eager_model.zero_grad(set_to_none=True), x)

    # 2) torch.compile (if available)
    if hasattr(torch, "compile"):
        compiled = torch.compile(eager_model)
        warmup(compiled, lambda: eager_model.zero_grad(set_to_none=True), x)
        results['torch.compile'] = peak_memory_mb(compiled, lambda: eager_model.zero_grad(set_to_none=True), x)
    else:
        print("torch.compile not available in this PyTorch build")

    # 3) Activation checkpointing
    warmup(ckpt_model, lambda: ckpt_model.zero_grad(set_to_none=True), x)
    results['Checkpointing'] = peak_memory_mb(ckpt_model, lambda: ckpt_model.zero_grad(set_to_none=True), x)

    print(f"FFN: dim={dim}, batch={batch}, seq={seq}")
    print("-" * 50)
    for name, mem in results.items():
        print(f"{name:20s}: {mem:8.2f} MB")
    print()
    print("torch.compile: better speed, moderate memory savings")
    print("Checkpointing: maximum memory savings, some speed cost")
else:
    print("[Run on GPU for comparison]")

FFN: dim=2048, batch=64, seq=128
--------------------------------------------------
Eager               :  2718.83 MB
torch.compile       :  2299.36 MB
Checkpointing       :  2567.79 MB

torch.compile: better speed, moderate memory savings
Checkpointing: maximum memory savings, some speed cost


### What Gets Recomputed at Each Budget Level?Based on real transformer results:| Budget | Recomputes | Saves | Memory ||--------|------------|-------|--------|| 1.0 (default) | Nothing extra | Everything | 100% || 0.7 | Pointwise ops (gelu, add, mul) | Matmuls, attention | ~85% || 0.5 | Pointwise + some matmuls | Attention (most expensive) | ~50% || 0.3 | Pointwise + most matmuls | Only attention | ~35% || 0.0 | Everything (like full AC) | Almost nothing | Minimum |**Key insight:** 50% memory reduction by recomputing only pointwise ops. Attention is expensive. Recompute it last.

---# 9b. External Libraries ReferenceBeyond PyTorch's built-in checkpointing, external libraries offer additional features.

In [94]:
# External Libraries: DeepSpeed and FairScale

print("""
DEEPSPEED ACTIVATION CHECKPOINTING
==================================
pip install deepspeed

Key features:
- Automatic checkpoint placement
- CPU offloading for extreme memory savings (offload activations to RAM!)
- Integrated with ZeRO optimizer stages
- Best for: very large models (billions of parameters)

Config in ds_config.json:
{
    "activation_checkpointing": {
        "partition_activations": true,
        "contiguous_memory_optimization": true,
        "cpu_checkpointing": true
    }
}


FAIRSCALE CHECKPOINT_WRAPPER
============================
pip install fairscale

Key features:
- Clean API: wrap modules directly with checkpoint_wrapper()
- Works well with FSDP (Fully Sharded Data Parallel)
- Simpler than DeepSpeed for many use cases
- Best for: medium-scale training

Usage:
from fairscale.nn.checkpoint import checkpoint_wrapper

layer = nn.Sequential(nn.Linear(1024, 1024), nn.ReLU())
checkpointed_layer = checkpoint_wrapper(layer)


TOOL SELECTION GUIDE
====================
- PyTorch native: Best for most cases, well-maintained
- DeepSpeed: Very large models, need CPU offloading
- FairScale: Using FSDP, prefer simpler API
""")


DEEPSPEED ACTIVATION CHECKPOINTING
pip install deepspeed

Key features:
- Automatic checkpoint placement
- CPU offloading for extreme memory savings (offload activations to RAM!)
- Integrated with ZeRO optimizer stages
- Best for: very large models (billions of parameters)

Config in ds_config.json:
{
    "activation_checkpointing": {
        "partition_activations": true,
        "contiguous_memory_optimization": true,
        "cpu_checkpointing": true
    }
}


FAIRSCALE CHECKPOINT_WRAPPER
pip install fairscale

Key features:
- Clean API: wrap modules directly with checkpoint_wrapper()
- Works well with FSDP (Fully Sharded Data Parallel)
- Simpler than DeepSpeed for many use cases
- Best for: medium-scale training

Usage:
from fairscale.nn.checkpoint import checkpoint_wrapper

layer = nn.Sequential(nn.Linear(1024, 1024), nn.ReLU())
checkpointed_layer = checkpoint_wrapper(layer)


TOOL SELECTION GUIDE
- PyTorch native: Best for most cases, well-maintained
- DeepSpeed: Very large mode

---# 10. Case Study: Activation Checkpointing in ResNet50ResNet50 is deep enough to benefit from checkpointing. It has 4 stages with multiple bottleneck blocks. Each block stores feature maps for backward.This section shows how to apply checkpointing to a real production model. You will see:1. How to modify torchvision's ResNet502. Memory comparison across checkpointing strategies3. A complete training loop with checkpointing

In [ ]:
# 10.1 ResNet50 Architecture Overview

import torch
import torch.nn as nn

try:
    import torchvision.models as models
except Exception as e:
    models = None
    print("torchvision is required for the ResNet50 case study.")
    print(f"Details: {type(e).__name__}: {e}")

if models is not None:
    # Load ResNet50 without downloading weights
    try:
        resnet50 = models.resnet50(weights=None)
    except TypeError:
        # Older torchvision
        resnet50 = models.resnet50(pretrained=False)

    print("ResNet50 Structure:")
    print("-" * 60)
    print(f"conv1:  1 conv layer")
    print(f"layer1: {len(resnet50.layer1)} Bottleneck blocks (64 -> 256 channels)")
    print(f"layer2: {len(resnet50.layer2)} Bottleneck blocks (128 -> 512 channels)")
    print(f"layer3: {len(resnet50.layer3)} Bottleneck blocks (256 -> 1024 channels)")
    print(f"layer4: {len(resnet50.layer4)} Bottleneck blocks (512 -> 2048 channels)")
    print(f"fc:     1 linear layer")
    print()
    print(
        f"Total Bottleneck blocks: {len(resnet50.layer1) + len(resnet50.layer2) + len(resnet50.layer3) + len(resnet50.layer4)}"
    )
    print()
    print("Each Bottleneck block has 3 conv layers + skip connection.")
    print("Feature maps grow larger in early layers, then shrink spatially.")
    print("layer3 often has the largest activation memory (1024 channels, moderate spatial size).")

In [ ]:
# 10.2 ResNet50 with Checkpointing: Three Strategies

import torch
import torch.nn as nn
from torch.utils.checkpoint import checkpoint

try:
    import torchvision.models as models
except Exception as e:
    models = None
    ResNet50Checkpointed = None
    print("torchvision is required for ResNet50Checkpointed.")
    print(f"Details: {type(e).__name__}: {e}")

if models is not None:
    class ResNet50Checkpointed(nn.Module):
        """ResNet50 with configurable checkpointing strategies."""

        def __init__(self, num_classes=1000, checkpoint_strategy='none'):
            """
            Args:
                checkpoint_strategy: 'none', 'per_stage', 'per_block', or 'aggressive'
            """
            super().__init__()

            # Load base ResNet50 without downloading weights
            try:
                base = models.resnet50(weights=None)
            except TypeError:
                base = models.resnet50(pretrained=False)

            # Copy all layers
            self.conv1 = base.conv1
            self.bn1 = base.bn1
            self.relu = base.relu
            self.maxpool = base.maxpool
            self.layer1 = base.layer1
            self.layer2 = base.layer2
            self.layer3 = base.layer3
            self.layer4 = base.layer4
            self.avgpool = base.avgpool
            self.fc = nn.Linear(2048, num_classes)

            self.checkpoint_strategy = checkpoint_strategy

        def _forward_stage(self, stage, x):
            """Forward through a stage (layer1, layer2, etc.)."""
            for block in stage:
                x = block(x)
            return x

        def forward(self, x):
            # Stem
            x = self.conv1(x)
            x = self.bn1(x)
            x = self.relu(x)
            x = self.maxpool(x)

            # NOTE: ResNet blocks include BatchNorm, which updates running stats in train mode.
            # Checkpointing recomputes forward in backward, which can update BN stats twice.
            # For strict parity, consider freezing BN stats (eval for BN) or using GroupNorm.

            if self.checkpoint_strategy == 'none':
                x = self.layer1(x)
                x = self.layer2(x)
                x = self.layer3(x)
                x = self.layer4(x)

            elif self.checkpoint_strategy == 'per_stage':
                x = checkpoint(lambda t: self._forward_stage(self.layer1, t), x, use_reentrant=False)
                x = checkpoint(lambda t: self._forward_stage(self.layer2, t), x, use_reentrant=False)
                x = checkpoint(lambda t: self._forward_stage(self.layer3, t), x, use_reentrant=False)
                x = checkpoint(lambda t: self._forward_stage(self.layer4, t), x, use_reentrant=False)

            elif self.checkpoint_strategy == 'per_block':
                for block in self.layer1:
                    x = checkpoint(block, x, use_reentrant=False)
                for block in self.layer2:
                    x = checkpoint(block, x, use_reentrant=False)
                for block in self.layer3:
                    x = checkpoint(block, x, use_reentrant=False)
                for block in self.layer4:
                    x = checkpoint(block, x, use_reentrant=False)

            elif self.checkpoint_strategy == 'aggressive':
                x = self.layer1(x)
                x = self.layer2(x)
                for block in self.layer3:
                    x = checkpoint(block, x, use_reentrant=False)
                for block in self.layer4:
                    x = checkpoint(block, x, use_reentrant=False)
            else:
                raise ValueError(f"Unknown checkpoint_strategy: {self.checkpoint_strategy}")

            # Head
            x = self.avgpool(x)
            x = torch.flatten(x, 1)
            x = self.fc(x)
            return x

    print("ResNet50Checkpointed created with 4 strategies:")
    print("-" * 60)
    print("'none':       No checkpointing (baseline)")
    print("'per_stage':  Checkpoint each of the 4 stages")
    print("'per_block':  Checkpoint each of the 16 bottleneck blocks")
    print("'aggressive': Only checkpoint layer3 and layer4 (best ROI)")

In [ ]:
# 10.3 Memory Comparison: ResNet50 Strategies

import torch
import torch.nn as nn
import time

def benchmark_resnet50(strategy, batch_size=32, image_size=224, num_iters=5):
    """Benchmark memory and time for a ResNet50 checkpointing strategy."""
    if ResNet50Checkpointed is None:
        raise RuntimeError("ResNet50Checkpointed is not available (torchvision import likely failed).")

    model = ResNet50Checkpointed(num_classes=1000, checkpoint_strategy=strategy).cuda()
    model.train()

    x = torch.randn(batch_size, 3, image_size, image_size, device="cuda")
    target = torch.randint(0, 1000, (batch_size,), device="cuda")
    criterion = nn.CrossEntropyLoss()

    # Warmup
    for _ in range(2):
        out = model(x)
        loss = criterion(out, target)
        loss.backward()
        model.zero_grad(set_to_none=True)

    # Measure memory + time
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.synchronize()

    start = time.time()
    for _ in range(num_iters):
        out = model(x)
        loss = criterion(out, target)
        loss.backward()
        model.zero_grad(set_to_none=True)
    torch.cuda.synchronize()

    elapsed = (time.time() - start) / num_iters * 1000  # ms/iter
    peak_mem = torch.cuda.max_memory_allocated() / 1e6  # MB

    del model, x, target
    torch.cuda.empty_cache()

    return peak_mem, elapsed

if torch.cuda.is_available() and ResNet50Checkpointed is not None:
    batch_size = 32
    image_size = 224

    print("ResNet50 Checkpointing Comparison")
    print(f"Batch size: {batch_size}, Image size: {image_size}x{image_size}")
    print("-" * 70)
    print(f"{'Strategy':<15} {'Peak Memory (MB)':<20} {'Time (ms)':<15} {'Mem Savings':<15}")
    print("-" * 70)

    strategies = ['none', 'aggressive', 'per_stage', 'per_block']
    results = {}

    for strategy in strategies:
        mem, time_ms = benchmark_resnet50(strategy, batch_size, image_size)
        results[strategy] = (mem, time_ms)

    baseline_mem = results['none'][0]
    for strategy in strategies:
        mem, time_ms = results[strategy]
        savings = (1 - mem / baseline_mem) * 100
        print(f"{strategy:<15} {mem:<20.2f} {time_ms:<15.2f} {savings:>6.1f}%")

    print("-" * 70)
    print("\nObservations:")
    print("- 'aggressive' gives best memory/speed balance (checkpoints only deep layers)")
    print("- 'per_block' gives maximum memory savings but highest overhead")
    print("- 'per_stage' is a middle ground")
else:
    print("[Run on GPU and ensure torchvision is installed for ResNet50 benchmarks]")

In [ ]:
# 10.4 Maximum Batch Size: ResNet50

import torch
import torch.nn as nn

def find_max_batch_resnet50(strategy, start=16, max_batch=256):
    """Find maximum batch size before OOM for ResNet50 (CUDA only)."""
    if not torch.cuda.is_available():
        raise RuntimeError("CUDA is required for find_max_batch_resnet50().")
    if ResNet50Checkpointed is None:
        raise RuntimeError("ResNet50Checkpointed is not available (torchvision import likely failed).")

    start = max(1, int(start))
    max_batch = max(start, int(max_batch))

    def can_run(batch_size):
        model = None
        x = None
        target = None
        out = None
        loss = None
        try:
            torch.cuda.empty_cache()
            model = ResNet50Checkpointed(checkpoint_strategy=strategy).cuda()
            model.train()
            x = torch.randn(batch_size, 3, 224, 224, device="cuda")
            target = torch.randint(0, 1000, (batch_size,), device="cuda")

            out = model(x)
            loss = nn.CrossEntropyLoss()(out, target)
            loss.backward()
            return True
        except RuntimeError as e:
            if "out of memory" in str(e).lower():
                return False
            raise
        finally:
            try:
                if model is not None:
                    model.zero_grad(set_to_none=True)
            except Exception:
                pass
            del model, x, target, out, loss
            torch.cuda.empty_cache()

    # Exponential search to find an upper bound, then binary search.
    max_working = 0
    batch = start
    while batch <= max_batch and can_run(batch):
        max_working = batch
        batch *= 2

    low = max_working + 1
    high = min(batch, max_batch)

    while low <= high:
        mid = (low + high) // 2
        if can_run(mid):
            max_working = mid
            low = mid + 1
        else:
            high = mid - 1

    return max_working

if torch.cuda.is_available() and ResNet50Checkpointed is not None:
    print("Finding maximum batch size for ResNet50 (224x224 images)")
    print("-" * 50)

    strategies = ['none', 'aggressive', 'per_block']
    baseline = None

    for strategy in strategies:
        max_batch = find_max_batch_resnet50(strategy)
        if baseline is None:
            baseline = max_batch
        improvement = (max_batch / baseline) if baseline else float('inf')
        print(f"{strategy:<15}: max batch = {max_batch:>3d} ({improvement:.2f}x)")

    print("-" * 50)
    print("\nWith checkpointing, you can often fit larger batches.")
else:
    print("[Run on GPU and ensure torchvision is installed to find max batch sizes]")

In [ ]:
# 10.5 Complete Training Loop: ResNet50 with Checkpointing

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

def train_resnet50_with_checkpointing(
    checkpoint_strategy='aggressive',
    batch_size=32,
    num_epochs=3,
    num_samples=256,  # Small for demo
    use_amp=True,
):
    """Complete training loop with checkpointing and mixed precision."""

    if ResNet50Checkpointed is None:
        raise RuntimeError("ResNet50Checkpointed is not available (torchvision import likely failed).")

    # Create model
    model = ResNet50Checkpointed(
        num_classes=10,  # Simplified for demo
        checkpoint_strategy=checkpoint_strategy,
    ).cuda()

    # Create synthetic dataset
    X = torch.randn(num_samples, 3, 224, 224)
    y = torch.randint(0, 10, (num_samples,))
    dataset = TensorDataset(X, y)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Optimizer and loss
    optimizer = optim.AdamW(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()
    scaler = torch.cuda.amp.GradScaler() if (use_amp and torch.cuda.is_available()) else None

    print(f"Training ResNet50 with '{checkpoint_strategy}' checkpointing")
    print(f"Batch size: {batch_size}, Mixed precision: {use_amp and scaler is not None}")
    print("-" * 50)

    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        num_batches = 0

        torch.cuda.reset_peak_memory_stats()

        for batch_x, batch_y in loader:
            batch_x = batch_x.cuda()
            batch_y = batch_y.cuda()

            optimizer.zero_grad(set_to_none=True)

            if scaler is not None:
                with torch.cuda.amp.autocast():
                    outputs = model(batch_x)
                    loss = criterion(outputs, batch_y)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

            epoch_loss += loss.item()
            num_batches += 1

        peak_mem = torch.cuda.max_memory_allocated() / 1e6
        avg_loss = epoch_loss / num_batches
        print(f"Epoch {epoch+1}/{num_epochs}: loss = {avg_loss:.4f}, peak memory = {peak_mem:.2f} MB")

    print("-" * 50)
    print("Training complete!")
    return model

if torch.cuda.is_available() and ResNet50Checkpointed is not None:
    model = train_resnet50_with_checkpointing(
        checkpoint_strategy='aggressive',
        batch_size=32,
        num_epochs=3,
        use_amp=True,
    )
else:
    print("[Run on GPU and ensure torchvision is installed for training demo]")

In [ ]:
# 10.6 Alternative: Using torchvision's Built-in Support

import torch
from torch.utils.checkpoint import checkpoint_sequential

try:
    import torchvision.models as models
except Exception as e:
    models = None
    print("torchvision is required for this alternative ResNet50 example.")
    print(f"Details: {type(e).__name__}: {e}")


def _checkpoint_sequential(mod, segments, x):
    """checkpoint_sequential signature varies across PyTorch versions."""
    try:
        return checkpoint_sequential(mod, segments, x, use_reentrant=False)
    except TypeError:
        return checkpoint_sequential(mod, segments, x)


def resnet50_with_sequential_checkpoint(num_segments=4):
    """Monkey-patch torchvision ResNet50 to checkpoint its stage blocks."""
    if models is None:
        raise RuntimeError("torchvision is not available")

    try:
        model = models.resnet50(weights=None)
    except TypeError:
        model = models.resnet50(pretrained=False)

    def checkpointed_forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        # Checkpoint each stage with user-controlled granularity.
        x = _checkpoint_sequential(self.layer1, min(num_segments, len(self.layer1)), x)
        x = _checkpoint_sequential(self.layer2, min(num_segments, len(self.layer2)), x)
        x = _checkpoint_sequential(self.layer3, min(num_segments, len(self.layer3)), x)
        x = _checkpoint_sequential(self.layer4, min(num_segments, len(self.layer4)), x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

    import types

    model.forward = types.MethodType(checkpointed_forward, model)
    return model

# Test it
if models is not None:
    model = resnet50_with_sequential_checkpoint(num_segments=4)
    print("Alternative approach: Monkey-patch torchvision ResNet50")
    print("-" * 60)
    print("Pros:")
    print("  - Uses official torchvision model")
    print("  - Simple implementation")
    print("  - Adjustable granularity via num_segments")
    print()
    print("Cons:")
    print("  - Less control over which blocks to checkpoint")
    print("  - Harder to switch strategies dynamically")
    print()
    print("Use the custom class (ResNet50Checkpointed) for flexible experiments.")

### Impressions/Conclusions (10: ResNet50 Case Study)ResNet50 is a perfect testbed for checkpointing because:- 16 bottleneck blocks across 4 stages- Deep enough to benefit, not so deep it is impractical- Widely used in productionKey findings:**Strategy Selection:**- `aggressive` (checkpoint layer3+layer4): Best ROI. These layers have the most parameters and largest feature maps.- `per_block`: Maximum memory savings (~40-50%), but highest compute overhead (~30-40%).- `per_stage`: Middle ground. Good for quick wins.**Practical Impact:**- 1.5-2x larger batch sizes possible- Combined with mixed precision: up to 3x improvement**BatchNorm Caveat:**ResNet blocks use BatchNorm. Checkpointing recomputes forward in backward, which can update BN running stats twice in `train()` mode. If you need strict parity, consider freezing BN stats (set BN layers to eval) or using GroupNorm.**Production Recommendations:**1. Start with `aggressive` strategy2. Combine with mixed precision (AMP)3. Profile before and after4. If still OOM, move to `per_block`The ResNet50Checkpointed class is a solid starting point. Copy it into your codebase and adapt it to your training setup and normalization strategy.

---# 11. ConclusionActivation checkpointing is a memory-compute trade-off. It trades extra forward passes for reduced memory usage. The trade-off is usually worth it.

In [ ]:
# 11.1 Key Takeaways

print("""
KEY TAKEAWAYS
=============

1. ACTIVATIONS DOMINATE MEMORY
   - Not parameters, not gradients
   - Scales with batch_size * depth * hidden_size
   - Peak memory occurs at start of backward pass

2. CHECKPOINTING TRADES COMPUTE FOR MEMORY
   - Save some activations (checkpoints)
   - Recompute others during backward pass
   - Typical: 30-50% memory savings, ~50% compute overhead

3. THE SQRT(N) RULE
   - Optimal checkpointing: O(sqrt(n)) memory for n layers
   - 100 layers -> ~10 checkpoints -> 10x memory reduction

4. PRACTICAL PATTERNS
   - checkpoint(): Single modules/functions
   - checkpoint_sequential(): Sequential models
   - Always use use_reentrant=False

5. ADVANCED: SELECTIVE ACTIVATION CHECKPOINT (SAC)
   - Fine-grained control: choose what to save vs recompute
   - Policy functions: MUST_SAVE for expensive ops, PREFER_RECOMPUTE for cheap
   - Sweet spot: save matmuls, recompute pointwise ops
   - Use aten.<op>.default for correct op matching

6. ADVANCED: MEMORY BUDGET API (torch.compile)
   - One line: torch._dynamo.config.activation_memory_budget = 0.5
   - Automatic pareto-optimal recomputation strategies
   - Budget 0 = full AC, Budget 1 = default compile

7. COMBINE WITH OTHER TECHNIQUES
   - Mixed precision: 2x memory from fp16
   - Gradient accumulation: Effective larger batches
   - DDP: Scale across GPUs
   - Together: Train 4-5x larger models
""")

In [ ]:
# 11.2 Decision Framework: When to Use Checkpointing

print("""
WHEN TO USE ACTIVATION CHECKPOINTING
====================================

USE IT WHEN:
- You are hitting OOM errors
- You want larger batch sizes
- Your model has 10+ layers
- Training time is less critical than memory
- You are training transformers or deep CNNs

SKIP IT WHEN:
- Model fits comfortably in memory
- Training time is the bottleneck (not memory)
- Model is shallow (< 5 layers)
- You need maximum training speed

QUICK DECISION TREE:

  OOM Error?
     |
     v
  YES --> Use checkpointing
     |
     v
  Want larger batches?
     |
     v
  YES --> Use checkpointing
     |
     v
  Model > 10 layers?
     |
     v
  YES --> Consider checkpointing
     |
     v
  NO --> Probably skip it
""")

### Final ImpressionsActivation checkpointing is not magic. It is a simple trade-off executed well.You now understand:- Why activations dominate memory (batch x depth x hidden)- How checkpointing works (recompute instead of store)- When to use it (memory-bound, deep models)- Basic implementation (checkpoint, checkpoint_sequential, use_reentrant=False)- Advanced control with SAC (choose exactly what to save)- Automatic optimization with Memory Budget API (one config line)The landscape of techniques:- **Eager**: Maximum speed, maximum memory- **torch.compile**: Free speedups, some automatic memory savings- **Memory Budget API**: Tunable compile-time optimization (0 to 1)- **Selective AC**: Manual control over save/recompute decisions- **Standard AC**: Maximum memory savings, ~50% compute overheadStart simple. Add complexity only when needed. Measure everything.Every large language model uses some form of activation checkpointing. Now you know exactly how it works and when to use each variant.Go train something bigger.

In [95]:
# 4.1 Deep Model Comparison: The Real Difference

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.checkpoint import checkpoint_sequential

class FFNBlock(nn.Module):
    """FFN block with 4x expansion - the memory-hungry part of transformers.
    
    The 4x expanded intermediate tensor is what makes activations large.
    This is why transformer FFN blocks benefit most from checkpointing.
    """
    def __init__(self, dim):
        super().__init__()
        self.fc1 = nn.Linear(dim, dim * 4)
        self.fc2 = nn.Linear(dim * 4, dim)
    
    def forward(self, x):
        return self.fc2(F.gelu(self.fc1(x)))

class DeepModelNoCheckpoint(nn.Module):
    def __init__(self, num_layers=10, hidden=1024):
        super().__init__()
        self.layers = nn.ModuleList([FFNBlock(hidden) for _ in range(num_layers)])
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

class DeepModelWithCheckpoint(nn.Module):
    """Uses checkpoint_sequential to checkpoint segments of layers.
    
    Key insight: checkpoint_sequential splits the model into `segments` chunks
    and only saves the input to each chunk. During backward, each chunk is
    recomputed from its saved input.
    
    Memory: O(segments) instead of O(num_layers)
    Compute: ~1 extra forward pass (recomputing during backward)
    """
    def __init__(self, num_layers=10, hidden=1024, num_segments=5):
        super().__init__()
        self.layers = nn.Sequential(*[FFNBlock(hidden) for _ in range(num_layers)])
        self.num_segments = num_segments
    
    def forward(self, x):
        return checkpoint_sequential(
            self.layers, self.num_segments, x, use_reentrant=False
        )

if torch.cuda.is_available():
    batch_size = 256
    hidden = 1024
    num_layers = 10
    num_segments = 5  # 5 segments of 2 layers each
    
    # Without checkpointing
    torch.cuda.empty_cache()
    model1 = DeepModelNoCheckpoint(num_layers, hidden).cuda()
    x1 = torch.randn(batch_size, hidden, device='cuda')
    
    torch.cuda.reset_peak_memory_stats()
    out1 = model1(x1)
    out1.sum().backward()
    mem_no_ckpt = torch.cuda.max_memory_allocated() / 1e6
    
    del model1, x1, out1
    torch.cuda.empty_cache()
    
    # With segment checkpointing
    model2 = DeepModelWithCheckpoint(num_layers, hidden, num_segments).cuda()
    x2 = torch.randn(batch_size, hidden, device='cuda')
    
    torch.cuda.reset_peak_memory_stats()
    out2 = model2(x2)
    out2.sum().backward()
    mem_ckpt = torch.cuda.max_memory_allocated() / 1e6
    
    # Calculate theoretical activation memory
    # Each FFNBlock saves the 4x expanded intermediate: batch * hidden * 4 * 4 bytes
    act_per_layer = batch_size * hidden * 4 * 4 / 1e6  # MB
    total_act = act_per_layer * num_layers
    saved_act = act_per_layer * (num_layers - num_segments)
    
    print(f"Config: {num_layers} FFN blocks (4x expansion), hidden={hidden}, batch={batch_size}")
    print(f"Segments: {num_segments} ({num_layers // num_segments} blocks each)")
    print()
    print(f"WITHOUT checkpointing: {mem_no_ckpt:.2f} MB")
    print(f"WITH checkpointing:    {mem_ckpt:.2f} MB")
    print(f"Memory saved: {mem_no_ckpt - mem_ckpt:.2f} MB ({(1 - mem_ckpt/mem_no_ckpt)*100:.1f}%)")
    print()
    print(f"Theoretical activation memory: {total_act:.1f} MB ({num_layers} layers x {act_per_layer:.1f} MB)")
    print(f"Expected savings: ~{saved_act:.1f} MB ({num_layers - num_segments} layers not stored)")
    print()
    print("The 4x expansion in FFN blocks is why transformer checkpointing works so well.")
    print("Simple Linear+ReLU layers have much smaller activations.")
else:
    print("[Run on GPU]")

Config: 10 FFN blocks (4x expansion), hidden=1024, batch=256
Segments: 5 (2 blocks each)

WITHOUT checkpointing: 2171.83 MB
WITH checkpointing:    2002.85 MB
Memory saved: 168.98 MB (7.8%)

Theoretical activation memory: 41.9 MB (10 layers x 4.2 MB)
Expected savings: ~21.0 MB (5 layers not stored)

The 4x expansion in FFN blocks is why transformer checkpointing works so well.
Simple Linear+ReLU layers have much smaller activations.


In [96]:
# Try with fewer segments (more aggressive checkpointing) and larger hidden
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.checkpoint import checkpoint_sequential

class FFNBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.fc1 = nn.Linear(dim, dim * 4)
        self.fc2 = nn.Linear(dim * 4, dim)
    
    def forward(self, x):
        return self.fc2(F.gelu(self.fc1(x)))

class DeepModelNoCheckpoint(nn.Module):
    def __init__(self, num_layers=10, hidden=1024):
        super().__init__()
        self.layers = nn.ModuleList([FFNBlock(hidden) for _ in range(num_layers)])
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

class DeepModelWithCheckpoint(nn.Module):
    def __init__(self, num_layers=10, hidden=1024, num_segments=2):
        super().__init__()
        self.layers = nn.Sequential(*[FFNBlock(hidden) for _ in range(num_layers)])
        self.num_segments = num_segments
    
    def forward(self, x):
        return checkpoint_sequential(
            self.layers, self.num_segments, x, use_reentrant=False
        )

# Test with different configs
configs = [
    (10, 1024, 5),  # Current: 10 layers, hidden=1024, 5 segments
    (10, 1024, 2),  # More aggressive: 2 segments
    (10, 2048, 2),  # Larger hidden
]

for num_layers, hidden, num_segments in configs:
    torch.cuda.empty_cache()
    
    batch_size = 256
    
    # Without checkpointing
    model1 = DeepModelNoCheckpoint(num_layers, hidden).cuda()
    x1 = torch.randn(batch_size, hidden, device='cuda')
    torch.cuda.reset_peak_memory_stats()
    out1 = model1(x1)
    out1.sum().backward()
    mem_no_ckpt = torch.cuda.max_memory_allocated() / 1e6
    del model1, x1, out1
    torch.cuda.empty_cache()
    
    # With checkpointing
    model2 = DeepModelWithCheckpoint(num_layers, hidden, num_segments).cuda()
    x2 = torch.randn(batch_size, hidden, device='cuda')
    torch.cuda.reset_peak_memory_stats()
    out2 = model2(x2)
    out2.sum().backward()
    mem_ckpt = torch.cuda.max_memory_allocated() / 1e6
    del model2, x2, out2
    
    saved = mem_no_ckpt - mem_ckpt
    pct = (1 - mem_ckpt/mem_no_ckpt)*100
    print(f"layers={num_layers}, hidden={hidden}, segments={num_segments}:")
    print(f"  No ckpt: {mem_no_ckpt:.0f} MB, With ckpt: {mem_ckpt:.0f} MB")
    print(f"  Saved: {saved:.0f} MB ({pct:.1f}%)")
    print()

layers=10, hidden=1024, segments=5:
  No ckpt: 2676 MB, With ckpt: 2357 MB
  Saved: 319 MB (11.9%)

layers=10, hidden=1024, segments=2:
  No ckpt: 2003 MB, With ckpt: 2003 MB
  Saved: 0 MB (0.0%)

layers=10, hidden=2048, segments=2:
  No ckpt: 4023 MB, With ckpt: 4023 MB
  Saved: 0 MB (0.0%)



In [97]:
# Debug: Check if checkpoint_sequential is working with 2 segments
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.checkpoint import checkpoint_sequential

class FFNBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.fc1 = nn.Linear(dim, dim * 4)
        self.fc2 = nn.Linear(dim * 4, dim)
    
    def forward(self, x):
        return self.fc2(F.gelu(self.fc1(x)))

batch_size = 256
hidden = 1024
num_layers = 10

# Fresh start
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

# Without checkpointing - measure at different points
layers_no_ckpt = nn.ModuleList([FFNBlock(hidden) for _ in range(num_layers)]).cuda()
x = torch.randn(batch_size, hidden, device='cuda')

# Forward pass
for layer in layers_no_ckpt:
    x = layer(x)
mem_after_forward = torch.cuda.memory_allocated() / 1e6

# Backward
x.sum().backward()
mem_peak = torch.cuda.max_memory_allocated() / 1e6

print(f"No checkpointing:")
print(f"  After forward: {mem_after_forward:.1f} MB")
print(f"  Peak memory:   {mem_peak:.1f} MB")

del layers_no_ckpt, x
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

# With checkpointing (2 segments)
layers_ckpt = nn.Sequential(*[FFNBlock(hidden) for _ in range(num_layers)]).cuda()
x = torch.randn(batch_size, hidden, device='cuda')

# Forward with checkpoint
out = checkpoint_sequential(layers_ckpt, 2, x, use_reentrant=False)
mem_after_forward_ckpt = torch.cuda.memory_allocated() / 1e6

# Backward
out.sum().backward()
mem_peak_ckpt = torch.cuda.max_memory_allocated() / 1e6

print(f"\nWith checkpointing (2 segments):")
print(f"  After forward: {mem_after_forward_ckpt:.1f} MB")
print(f"  Peak memory:   {mem_peak_ckpt:.1f} MB")

print(f"\nSavings after forward: {mem_after_forward - mem_after_forward_ckpt:.1f} MB")
print(f"Savings peak:          {mem_peak - mem_peak_ckpt:.1f} MB")

No checkpointing:
  After forward: 1689.1 MB
  Peak memory:   1935.7 MB

With checkpointing (2 segments):
  After forward: 1710.1 MB
  Peak memory:   2002.9 MB

Savings after forward: -21.0 MB
Savings peak:          -67.1 MB


In [98]:
# Test with varying segment counts to find the sweet spot
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.checkpoint import checkpoint_sequential

class FFNBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.fc1 = nn.Linear(dim, dim * 4)
        self.fc2 = nn.Linear(dim * 4, dim)
    
    def forward(self, x):
        return self.fc2(F.gelu(self.fc1(x)))

batch_size = 256
hidden = 1024
num_layers = 10

# Baseline - no checkpointing
torch.cuda.empty_cache()
model_base = nn.ModuleList([FFNBlock(hidden) for _ in range(num_layers)]).cuda()
x = torch.randn(batch_size, hidden, device='cuda')
torch.cuda.reset_peak_memory_stats()
for layer in model_base:
    x = layer(x)
x.sum().backward()
baseline_mem = torch.cuda.max_memory_allocated() / 1e6
del model_base, x
torch.cuda.empty_cache()

print(f"Baseline (no checkpoint): {baseline_mem:.0f} MB")
print()
print(f"{'Segments':<10} {'Layers/Seg':<12} {'Peak MB':<12} {'Saved MB':<12} {'Savings %':<10}")
print("-" * 60)

for num_segments in [2, 3, 4, 5, 10]:  # 10 segments = per-layer checkpointing
    torch.cuda.empty_cache()
    model = nn.Sequential(*[FFNBlock(hidden) for _ in range(num_layers)]).cuda()
    x = torch.randn(batch_size, hidden, device='cuda')
    torch.cuda.reset_peak_memory_stats()
    out = checkpoint_sequential(model, num_segments, x, use_reentrant=False)
    out.sum().backward()
    mem = torch.cuda.max_memory_allocated() / 1e6
    
    saved = baseline_mem - mem
    pct = (saved / baseline_mem) * 100 if baseline_mem > 0 else 0
    layers_per_seg = num_layers / num_segments
    
    print(f"{num_segments:<10} {layers_per_seg:<12.1f} {mem:<12.0f} {saved:<12.0f} {pct:<10.1f}")
    del model, x, out
    
print()
print("Observation: More segments = more memory savings (to a point).")

Baseline (no checkpoint): 2608 MB

Segments   Layers/Seg   Peak MB      Saved MB     Savings % 
------------------------------------------------------------
2          5.0          2674         -66          -2.5      
3          3.3          2674         -66          -2.5      
4          2.5          2674         -66          -2.5      
5          2.0          2674         -66          -2.5      
10         1.0          2674         -66          -2.5      

Observation: More segments = more memory savings (to a point).


In [99]:
# Clean test - completely fresh state for each measurement
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.checkpoint import checkpoint_sequential
import gc

class FFNBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.fc1 = nn.Linear(dim, dim * 4)
        self.fc2 = nn.Linear(dim * 4, dim)
    
    def forward(self, x):
        return self.fc2(F.gelu(self.fc1(x)))

def measure_no_checkpoint(num_layers, hidden, batch_size):
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    
    model = nn.ModuleList([FFNBlock(hidden) for _ in range(num_layers)]).cuda()
    x = torch.randn(batch_size, hidden, device='cuda')
    
    for layer in model:
        x = layer(x)
    x.sum().backward()
    
    peak = torch.cuda.max_memory_allocated() / 1e6
    
    del model, x
    gc.collect()
    torch.cuda.empty_cache()
    
    return peak

def measure_with_checkpoint(num_layers, hidden, batch_size, num_segments):
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    
    model = nn.Sequential(*[FFNBlock(hidden) for _ in range(num_layers)]).cuda()
    x = torch.randn(batch_size, hidden, device='cuda')
    
    out = checkpoint_sequential(model, num_segments, x, use_reentrant=False)
    out.sum().backward()
    
    peak = torch.cuda.max_memory_allocated() / 1e6
    
    del model, x, out
    gc.collect()
    torch.cuda.empty_cache()
    
    return peak

# Config
num_layers = 10
hidden = 1024
batch_size = 256

print(f"Config: {num_layers} FFN blocks, hidden={hidden}, batch={batch_size}")
print()

# Measure baseline
baseline = measure_no_checkpoint(num_layers, hidden, batch_size)
print(f"No checkpointing: {baseline:.0f} MB")
print()

# Measure with different segment counts
for segs in [2, 5, 10]:
    mem = measure_with_checkpoint(num_layers, hidden, batch_size, segs)
    saved = baseline - mem
    pct = (saved / baseline) * 100
    print(f"{segs} segments: {mem:.0f} MB (saved {saved:.0f} MB, {pct:.1f}%)")

Config: 10 FFN blocks, hidden=1024, batch=256

No checkpointing: 2674 MB

2 segments: 2674 MB (saved 0 MB, 0.0%)
5 segments: 2674 MB (saved 0 MB, 0.0%)
10 segments: 2674 MB (saved 0 MB, 0.0%)


In [103]:
# Diagnostic: Check if checkpointing is actually working
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.checkpoint import checkpoint

class FFNBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.fc1 = nn.Linear(dim, dim * 4)
        self.fc2 = nn.Linear(dim * 4, dim)
    
    def forward(self, x):
        return self.fc2(F.gelu(self.fc1(x)))

num_layers = 12
hidden = 2048
batch_size = 128

# Test 1: Without checkpointing - measure memory after forward
torch.cuda.empty_cache()
blocks1 = nn.ModuleList([FFNBlock(hidden) for _ in range(num_layers)]).cuda()
x1 = torch.randn(batch_size, hidden, device='cuda', requires_grad=True)

mem_before = torch.cuda.memory_allocated() / 1e6

# Forward without checkpoint
out = x1
for block in blocks1:
    out = block(out)

torch.cuda.synchronize()
mem_after_fwd_no_ckpt = torch.cuda.memory_allocated() / 1e6
activations_no_ckpt = mem_after_fwd_no_ckpt - mem_before

print("=== WITHOUT Checkpointing ===")
print(f"Before forward:     {mem_before:.1f} MB")
print(f"After forward:      {mem_after_fwd_no_ckpt:.1f} MB")
print(f"Activation memory:  {activations_no_ckpt:.1f} MB")

del blocks1, x1, out
torch.cuda.empty_cache()

# Test 2: With checkpointing - measure memory after forward
blocks2 = nn.ModuleList([FFNBlock(hidden) for _ in range(num_layers)]).cuda()
x2 = torch.randn(batch_size, hidden, device='cuda', requires_grad=True)

mem_before2 = torch.cuda.memory_allocated() / 1e6

# Forward WITH checkpoint
out2 = x2
for block in blocks2:
    out2 = checkpoint(block, out2, use_reentrant=False)

torch.cuda.synchronize()
mem_after_fwd_ckpt = torch.cuda.memory_allocated() / 1e6
activations_ckpt = mem_after_fwd_ckpt - mem_before2

print("\n=== WITH Checkpointing ===")
print(f"Before forward:     {mem_before2:.1f} MB")
print(f"After forward:      {mem_after_fwd_ckpt:.1f} MB")
print(f"Activation memory:  {activations_ckpt:.1f} MB")

print(f"\n=== Comparison ===")
print(f"Activation savings: {activations_no_ckpt - activations_ckpt:.1f} MB")
print(f"Reduction:          {(1 - activations_ckpt/activations_no_ckpt)*100:.1f}%")

=== WITHOUT Checkpointing ===
Before forward:     5236.6 MB
After forward:      5336.4 MB
Activation memory:  99.7 MB

=== WITH Checkpointing ===
Before forward:     5357.4 MB
After forward:      5235.7 MB
Activation memory:  -121.7 MB

=== Comparison ===
Activation savings: 221.4 MB
Reduction:          222.0%


In [104]:
# 4.1 Deep Model Comparison: The Real Difference

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.checkpoint import checkpoint

class FFNBlock(nn.Module):
    """FFN block with 4x expansion - the memory-hungry part of transformers."""
    def __init__(self, dim):
        super().__init__()
        self.fc1 = nn.Linear(dim, dim * 4)
        self.fc2 = nn.Linear(dim * 4, dim)
    
    def forward(self, x):
        return self.fc2(F.gelu(self.fc1(x)))

if torch.cuda.is_available():
    num_layers = 12
    hidden = 2048
    batch_size = 128
    
    # ===== Test 1: Without checkpointing =====
    torch.cuda.empty_cache()
    blocks1 = nn.ModuleList([FFNBlock(hidden) for _ in range(num_layers)]).cuda()
    x1 = torch.randn(batch_size, hidden, device='cuda', requires_grad=True)
    
    mem_before1 = torch.cuda.memory_allocated() / 1e6
    
    out = x1
    for block in blocks1:
        out = block(out)
    
    torch.cuda.synchronize()
    mem_after_fwd1 = torch.cuda.memory_allocated() / 1e6
    activations1 = mem_after_fwd1 - mem_before1
    
    # Measure peak during backward
    torch.cuda.reset_peak_memory_stats()
    out.sum().backward()
    peak1 = torch.cuda.max_memory_allocated() / 1e6
    
    del blocks1, x1, out
    torch.cuda.empty_cache()
    
    # ===== Test 2: With checkpointing =====
    blocks2 = nn.ModuleList([FFNBlock(hidden) for _ in range(num_layers)]).cuda()
    x2 = torch.randn(batch_size, hidden, device='cuda', requires_grad=True)
    
    mem_before2 = torch.cuda.memory_allocated() / 1e6
    
    out2 = x2
    for block in blocks2:
        out2 = checkpoint(block, out2, use_reentrant=False)
    
    torch.cuda.synchronize()
    mem_after_fwd2 = torch.cuda.memory_allocated() / 1e6
    activations2 = mem_after_fwd2 - mem_before2
    
    # Measure peak during backward
    torch.cuda.reset_peak_memory_stats()
    out2.sum().backward()
    peak2 = torch.cuda.max_memory_allocated() / 1e6
    
    print(f"Config: {num_layers} FFN blocks (4x expansion), hidden={hidden}, batch={batch_size}")
    print("=" * 65)
    print()
    print(f"{'Metric':<30} {'No Ckpt':>12} {'With Ckpt':>12} {'Saved':>10}")
    print("-" * 65)
    print(f"{'Activations after forward:':<30} {activations1:>10.1f} MB {activations2:>10.1f} MB {activations1-activations2:>8.1f} MB")
    print(f"{'Peak during backward:':<30} {peak1:>10.1f} MB {peak2:>10.1f} MB {peak1-peak2:>8.1f} MB")
    print()
    print("KEY INSIGHT:")
    print("- Checkpointing eliminates activation storage during forward pass")
    print("- Peak memory during backward may be similar (recomputation recreates activations)")
    print("- The WIN is being able to run larger batches that would otherwise OOM")
    print()
    print("In practice: checkpointing lets you increase batch size until you hit")
    print("the new (lower) memory ceiling, then backward peak becomes the limit.")
else:
    print("[Run on GPU]")

Config: 12 FFN blocks (4x expansion), hidden=2048, batch=128

Metric                              No Ckpt    With Ckpt      Saved
-----------------------------------------------------------------
Activations after forward:          113.2 MB    -1880.7 MB   1993.9 MB
Peak during backward:              8465.2 MB     6840.5 MB   1624.7 MB

KEY INSIGHT:
- Checkpointing eliminates activation storage during forward pass
- Peak memory during backward may be similar (recomputation recreates activations)
- The WIN is being able to run larger batches that would otherwise OOM

In practice: checkpointing lets you increase batch size until you hit
the new (lower) memory ceiling, then backward peak becomes the limit.


In [105]:
# 4.1 Deep Model Comparison: The Real Difference

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.checkpoint import checkpoint

class FFNBlock(nn.Module):
    """FFN block with 4x expansion - the memory-hungry part of transformers."""
    def __init__(self, dim):
        super().__init__()
        self.fc1 = nn.Linear(dim, dim * 4)
        self.fc2 = nn.Linear(dim * 4, dim)
    
    def forward(self, x):
        return self.fc2(F.gelu(self.fc1(x)))

def measure_training_memory(blocks, x, use_checkpoint=False):
    """Measure peak memory during a full forward+backward pass."""
    torch.cuda.reset_peak_memory_stats()
    
    out = x
    for block in blocks:
        if use_checkpoint:
            out = checkpoint(block, out, use_reentrant=False)
        else:
            out = block(out)
    
    out.sum().backward()
    
    for block in blocks:
        block.zero_grad(set_to_none=True)
    
    return torch.cuda.max_memory_allocated() / 1e6

if torch.cuda.is_available():
    num_layers = 12
    hidden = 2048
    batch_size = 128
    
    # Create model and input (shared for fair comparison)
    torch.cuda.empty_cache()
    blocks = nn.ModuleList([FFNBlock(hidden) for _ in range(num_layers)]).cuda()
    
    # Warmup
    x = torch.randn(batch_size, hidden, device='cuda', requires_grad=True)
    _ = measure_training_memory(blocks, x, use_checkpoint=False)
    
    # Measure WITHOUT checkpointing
    torch.cuda.empty_cache()
    x = torch.randn(batch_size, hidden, device='cuda', requires_grad=True)
    mem_no_ckpt = measure_training_memory(blocks, x, use_checkpoint=False)
    
    # Measure WITH checkpointing (same model)
    torch.cuda.empty_cache()
    x = torch.randn(batch_size, hidden, device='cuda', requires_grad=True)
    mem_ckpt = measure_training_memory(blocks, x, use_checkpoint=True)
    
    saved = mem_no_ckpt - mem_ckpt
    pct = (saved / mem_no_ckpt) * 100
    
    print(f"Config: {num_layers} FFN blocks (4x expansion), hidden={hidden}, batch={batch_size}")
    print("=" * 60)
    print()
    print(f"Peak memory WITHOUT checkpointing: {mem_no_ckpt:,.0f} MB")
    print(f"Peak memory WITH checkpointing:    {mem_ckpt:,.0f} MB")
    print(f"Memory saved:                      {saved:,.0f} MB ({pct:.1f}%)")
    print()
    print("WHY THIS MATTERS:")
    print(f"  - You could increase batch size by ~{pct:.0f}% before hitting OOM")
    print(f"  - Or train a model ~{pct:.0f}% larger with the same GPU")
    print()
    print("TRADE-OFF: ~50% more compute (one extra forward pass per block)")
else:
    print("[Run on GPU]")

Config: 12 FFN blocks (4x expansion), hidden=2048, batch=128

Peak memory WITHOUT checkpointing: 10,066 MB
Peak memory WITH checkpointing:    10,066 MB
Memory saved:                      0 MB (0.0%)

WHY THIS MATTERS:
  - You could increase batch size by ~0% before hitting OOM
  - Or train a model ~0% larger with the same GPU

TRADE-OFF: ~50% more compute (one extra forward pass per block)


In [106]:
# Fresh models for each test - avoid CUDA memory pool interference
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.checkpoint import checkpoint
import gc

class FFNBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.fc1 = nn.Linear(dim, dim * 4)
        self.fc2 = nn.Linear(dim * 4, dim)
    
    def forward(self, x):
        return self.fc2(F.gelu(self.fc1(x)))

num_layers = 12
hidden = 2048
batch_size = 128

# ===== Test 1: WITHOUT checkpointing (fresh model) =====
gc.collect()
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

model1 = nn.ModuleList([FFNBlock(hidden) for _ in range(num_layers)]).cuda()
x1 = torch.randn(batch_size, hidden, device='cuda', requires_grad=True)

out = x1
for block in model1:
    out = block(out)
out.sum().backward()

mem_no_ckpt = torch.cuda.max_memory_allocated() / 1e6

# Full cleanup
del model1, x1, out
gc.collect()
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

# ===== Test 2: WITH checkpointing (fresh model) =====
model2 = nn.ModuleList([FFNBlock(hidden) for _ in range(num_layers)]).cuda()
x2 = torch.randn(batch_size, hidden, device='cuda', requires_grad=True)

out2 = x2
for block in model2:
    out2 = checkpoint(block, out2, use_reentrant=False)
out2.sum().backward()

mem_ckpt = torch.cuda.max_memory_allocated() / 1e6

saved = mem_no_ckpt - mem_ckpt
pct = (saved / mem_no_ckpt) * 100 if mem_no_ckpt > 0 else 0

print(f"Config: {num_layers} FFN blocks (4x expansion), hidden={hidden}, batch={batch_size}")
print("=" * 60)
print()
print(f"Peak memory WITHOUT checkpointing: {mem_no_ckpt:,.0f} MB")
print(f"Peak memory WITH checkpointing:    {mem_ckpt:,.0f} MB")
print(f"Memory saved:                      {saved:,.0f} MB ({pct:.1f}%)")

Config: 12 FFN blocks (4x expansion), hidden=2048, batch=128

Peak memory WITHOUT checkpointing: 11,679 MB
Peak memory WITH checkpointing:    10,329 MB
Memory saved:                      1,350 MB (11.6%)
